# XgBoost 

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
import numpy as np
import pandas as pd
import os
nltk.download('vader_lexicon')
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/yeetusonthefetus/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/yeetusonthefetus/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
combined_df = pd.read_csv('../datasets/combined_df.csv')

In [4]:
# Tokenize and clean text data (you can use regular expressions for more advanced cleaning)
combined_df['post'] = combined_df['post'].apply(lambda x: x.lower())  # Convert to lowercase

In [5]:
nltk.download('punkt') 

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/yeetusonthefetus/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/yeetusonthefetus/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Preprocessing data


In [7]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

def preprocess_text(text):
    # Tokenization and removing punctuation
    words = nltk.word_tokenize(text)
    words = [word for word in words if word.isalnum()]
    
    # Removing stop words and lemmatization
    stop_words = set(stopwords.words("english"))
    words = [WordNetLemmatizer().lemmatize(word) for word in words if word not in stop_words]
    
    return " ".join(words)
    
combined_df['cleaned_posts'] = combined_df['post'].apply(preprocess_text)


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/yeetusonthefetus/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_features=1000)  # You can adjust the number of features
X_tfidf = tfidf_vectorizer.fit_transform(combined_df['cleaned_posts'])


In [9]:
# You can load pre-trained word embeddings using libraries like gensim
from gensim.models import Word2Vec

# Train Word2Vec model on your preprocessed text data
word2vec_model = Word2Vec(sentences=[text.split() for text in combined_df['cleaned_posts']], vector_size=100, window=5, min_count=1, sg=1)


In [10]:
# Create a function to generate Word2Vec embeddings for a given text
def get_word2vec_features(text, model):
    words = text.split()
    # Initialize an empty vector
    feature_vector = np.zeros((model.vector_size,), dtype="float32")
    num_words = 0
    # Iterate over each word in the text
    for word in words:
        if word in model.wv:
            num_words += 1
            # Add the word's vector to the feature_vector
            feature_vector = np.add(feature_vector, model.wv[word])
    # Divide the result by the number of words to get the average
    if num_words != 0:
        feature_vector = np.divide(feature_vector, num_words)
    return feature_vector

# Generate Word2Vec features for each post in the DataFrame
combined_df['word2vec_features'] = combined_df['cleaned_posts'].apply(lambda post: get_word2vec_features(post, word2vec_model))


In [11]:
analyzer = SentimentIntensityAnalyzer()

def sentiment_scores(post):
    sentiment = analyzer.polarity_scores(post)
    return sentiment['compound']

combined_df['sentiment'] = combined_df['post'].apply(sentiment_scores)


In [12]:
# Concatenate all features
from scipy.sparse import hstack

X_tfidf_array = X_tfidf.toarray()  # Convert TF-IDF matrix to a dense array
word2vec_arrays = np.array(list(combined_df['word2vec_features']))

In [13]:
print(combined_df[['sentiment']].values.shape)
print(X_tfidf_array.shape)
print(word2vec_arrays.shape)

(374499, 1)
(374499, 1000)
(374499, 100)


In [14]:
sentiment_array = combined_df[['sentiment']].values

# Flatten the sentiment_array
sentiment_array_flat = sentiment_array.flatten()

# Convert sentiment_array_flat to a 2D array
sentiment_array_2d = sentiment_array_flat.reshape(-1, 1)

In [15]:
from gensim import corpora, models


documents = combined_df['cleaned_posts'].str.split()  # Split the preprocessed text into words

# Create a dictionary mapping words to IDs
dictionary = corpora.Dictionary(documents)

# Create a bag of words corpus
corpus = [dictionary.doc2bow(doc) for doc in documents]

# Train the LDA model
lda_model = models.LdaModel(corpus, num_topics=10, id2word=dictionary, passes=15)  # Adjust num_topics as needed

In [16]:
topic_distributions = [lda_model[doc] for doc in corpus]

In [17]:
num_topics = max(len(topic) for topic in topic_distributions)
topic_features = np.zeros((len(topic_distributions), num_topics))  # num_topics is the number of topics in your LDA model

for i, doc_topics in enumerate(topic_distributions):
    for topic, weight in doc_topics:
        topic_features[i, topic] = weight

In [18]:
# Combine the topic modeling output with X_combined
shape_tfidf = X_tfidf_array.shape
shape_word2vec = word2vec_arrays.shape
shape_topic = topic_features.shape  # Shape of the topic modeling output

# Create an empty array to accommodate the combined data
X_combined = np.empty((shape_tfidf[0], shape_tfidf[1] + shape_word2vec[1] + shape_topic[1] + 1))

# Copy data from the original arrays to the combined array
X_combined[:, :shape_tfidf[1]] = X_tfidf_array
X_combined[:, shape_tfidf[1]:shape_tfidf[1] + shape_word2vec[1]] = word2vec_arrays
X_combined[:, shape_tfidf[1] + shape_word2vec[1]:shape_tfidf[1] + shape_word2vec[1] + shape_topic[1]] = topic_features
X_combined[:, -1] = sentiment_array_flat


In [19]:
from sklearn.model_selection import train_test_split

X = combined_df.drop(['subreddit', 'post'],  axis=1)  # Features
y = combined_df['subreddit']  # Target


In [22]:

X_train, X_test, y_train, y_test = train_test_split(X_combined  , y, test_size=0.2, random_state=42)



In [23]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Step 1: Encode the categorical target variable (y_train) using OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)
y_train_encoded = encoder.fit_transform(y_train.values.reshape(-1, 1))

# Step 2: Flatten the encoded labels
y_train_encoded = y_train_encoded.argmax(axis=1)
y_test_encoded = encoder.transform(y_test.values.reshape(-1, 1)).argmax(axis=1)

## Training XGBoost on the data

In [28]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Assuming you have X_combined as your feature matrix and y as your target variable



# Create the XGBoost DMatrix for training and testing
dtrain = xgb.DMatrix(X_train, label=y_train_encoded)
dtest = xgb.DMatrix(X_test, label=y_test_encoded)
# Calculate the class imbalance ratio
positive_class_count = sum(y_train_encoded)
negative_class_count = len(y_train_encoded) - positive_class_count
class_imbalance_ratio = negative_class_count / positive_class_count

# Define XGBoost parameters
params = {
    'objective': 'multi:softmax',  # For multi-class classification
    'num_class': 11,  # Number of classes
    'eval_metric': 'mlogloss',
    'eta': 0.05,  # Learning rate
    'max_depth': None,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'verbose':2,
    'seed': 42
}

# Set class weights
class_weights = {0: 1, 1: 10}  # Class 0: 1, Class 1: 10

# Set "scale_pos_weight" based on class balance
pos_weight = sum(class_weights.values()) / sum(class_weights.keys())

# Train XGBoost model
model = xgb.XGBClassifier(scale_pos_weight=pos_weight, class_weight=class_weights)
model.fit(X_train, y_train_encoded)


/Users/yeetusonthefetus/iiit/SMAI/project-boys/.venv/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [13:00:50] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "class_weight", "scale_pos_weight" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              class_weight={0: 1, 1: 10}, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, ...)

# Results

In [29]:

# Make predictions on the test data
y_pred = model.predict(X_test)

# Evaluate the model's accuracy
accuracy = accuracy_score(y_test_encoded, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')


Accuracy: 71.27%


: 